In [ ]:
from IPython.display import clear_output
import os
import re
import shutil
import glob
import joe_penna_dreambooth_config

class copy_and_name_checkpoints:
    active_config_path = "./joepenna-dreambooth-configs/active-config.json"
    trained_models_path = "./trained_models"

    def __init__(self):
        pass

    def execute(self):
        clear_output()
        config = joe_penna_dreambooth_config.parse_config_file(self.active_config_path)

        if config is not None:

            if not os.path.exists(self.trained_models_path):
                os.mkdir(self.trained_models_path)

            # copy the config file to the model directory as well
            shutil.copy(self.active_config_path, f"{self.trained_models_path}/{config.config_file_name}")

            latest_training_directory = glob.glob("./logs/*")[-1]

            # Gather, name, and move the checkpoint files.
            checkpoint_paths = []
            steps = []
            if config.save_every_x_steps == 0:
                checkpoint_paths.append(f"{latest_training_directory}/checkpoints/last.ckpt")
                steps.append(str(config.max_training_steps))
            else:
                checkpoints_directory = f"{latest_training_directory}/checkpoints/trainstep_checkpoints"
                file_paths = glob.glob(f"{checkpoints_directory}/*")

                for i, original_file_name in enumerate(file_paths):
                    # Remove the "epoch=000000-step=0000" text
                    checkpoint_steps = re.sub(checkpoints_directory + "/epoch=\d{6}-step=0*", "", original_file_name)

                    # Remove the .ckpt
                    checkpoint_steps = checkpoint_steps.replace(".ckpt", "")

                    checkpoint_paths.append(f"{original_file_name}")
                    steps.append(checkpoint_steps)

            for i, training_steps in enumerate(steps):
                # Setup the filenames
                original_file_name = checkpoint_paths[i]
                file_name = config.createCheckpointFileName(training_steps)

                # Move the checkpoint
                shutil.move(original_file_name, os.path.join(self.trained_models_path, file_name))

            print("✅ Download your trained model(s) from the 'trained_models' folder and use in your favorite Stable Diffusion repo!")

copy_and_name_checkpoints = copy_and_name_checkpoints()